In [64]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import numpy as np 
from time import sleep
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import random as ra
import asyncio
import aiohttp


In [56]:
# Función para extraer el código de la estación
def extraer_codigo_estacion():
    driver = webdriver.Chrome(executable_path='/ruta/a/chromedriver')
    url = "https://www.wunderground.com/weather/es/alcorcon"
    driver.get(url)
    driver.maximize_window()
    
    # Esperar a que el iframe de las cookies esté presente y cambiar al iframe
    iframe_cookies = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(('xpath', '//*[@id="sp_message_iframe_1165301"]'))
    )
    driver.switch_to.frame(iframe_cookies)
    
    # Intentar rechazar las cookies
    try:
        driver.implicitly_wait(5)
        driver.find_element('css selector', "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()
    except:
        print("No encuentro el botón de cookies")
    
    # Volver al contenido principal
    driver.switch_to.default_content()
    sleep(3)
    
    # Extraer el código de la estación desde la URL del enlace de la estación
    datos = driver.find_element('css selector', "#inner-content > div.region-content-top > lib-city-header > div:nth-child(1) > div > div > a.station-name")
    url = datos.get_attribute("href")
    url_temp = url.split("?")
    code_station = url_temp[0].rsplit('/', 1)[-1]
    print(code_station)
    driver.quit()
       # Devolver el código de la estación
    return code_station


In [58]:
print(code_station)

['https://www.wunderground.com/dashboard/pws', 'IALCOR16']


In [65]:

# Función asíncrona para solicitar datos históricos
async def fecha_historica_data(session, url):
    async with session.get(url) as response:
        return await response.text()

#async def fetch_history_data(session, url): Esta función usa aiohttp para hacer una solicitud GET a una URL
# y devuelve el contenido HTML.

In [67]:
fecha_historica_data

<function __main__.fecha_historica_data(session, url)>

In [73]:

# Función principal para extraer datos históricos
async def extraer_datos_historicos(code_station):
    url_wunder = f"https://www.wunderground.com/history/daily/es/alcorcon/date/{code_station}"
    async with aiohttp.ClientSession() as session:
        html = await fecha_historica_data(session, url_wunder)
        soup = BeautifulSoup(html, 'html.parser')

        # Extraer la tabla de datos de la página
        tabla = soup.select_one("#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div > div > table > tbody")
        print(tabla.text)

        # Procesar los datos de la tabla
        datos = tabla.text.split('\n')
        print(datos)
        df = pd.DataFrame(datos)
        print(df)
#async def extraer_datos_historicos(code_station): Esta función principal usa fecha_history_data
#para obtener los datos HTML de la página histórica y luego usa BeautifulSoup para analizar el HTML.

async def main():
    getattr


#async def main(): Esta es la función principal que se encarga de coordinar las otras funciones. Primero llama a extraer_codigo_estacion() 
#para obtener el código de la estación y luego a extraer_datos_historicos(code_station) para obtener los datos históricos.

In [74]:
if __name__ == "__main__":
    asyncio.run(main())                        # inciar la funcion main

,municipio,latitud,longitud
0,acebeda-la,41.086958,-3.624399
1,ajalvir,40.534230,-3.480782
2,alameda-del-valle,40.917718,-3.843822
3,alamo-el,40.230730,-3.990589
4,alcala-de-henares,40.481840,-3.364497
...,...,...,...
174,villar-del-olmo,40.336447,-3.235562
175,villarejo-de-salvanes,40.168358,-3.273876
176,villaviciosa-de-odon,40.357379,-3.900233
177,villavieja-del-lozoya,41.006362,-3.671629
